In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
path='/content/drive/MyDrive/PRNN/Assign_3/animal/animals10_data.npz'
data = np.load(path)

# Load the data from the .npz file
image_data = np.load(path)
image_data=image_data['data']
# Extract images and labels
labels = data['labels']

In [ ]:
# Define the sizes for training, validation, and testing sets
train_size = int(0.2 * len(image_data))
val_size = int(0.7 * len(image_data))
test_size = len(image_data) - train_size - val_size

# Shuffle the data
indices = np.random.permutation(len(image_data))
image_data_shuffled = image_data[indices]
labels_shuffled = labels[indices]

# Split the data into training, validation, and testing sets
x_train = image_data_shuffled[:train_size]
y_train = labels_shuffled[:train_size]

x_val = image_data_shuffled[train_size:train_size+val_size]
y_val = labels_shuffled[train_size:train_size+val_size]

x_test = image_data_shuffled[train_size+val_size:]
y_test = labels_shuffled[train_size+val_size:]

# Print the shapes of the split sets
print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_val:", x_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)


Shape of x_train: (5235, 4096)
Shape of y_train: (5235,)
Shape of x_val: (18325, 4096)
Shape of y_val: (18325,)
Shape of x_test: (2619, 4096)
Shape of y_test: (2619,)


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU")


GPU is not available, using CPU


In [ ]:
# impurity function
#Entropy  formula used= summation of -pj*log(pj) over j=1 to k
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

In [ ]:
def gini(y):
    unique,label_count = np.unique(y,return_counts = True)
    ps = label_count/np.sum(label_count)
    return 1 - np.sum(ps**2)

In [ ]:
#defining a node

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [ ]:
#Decision Tree

class DecisionTree:
    def __init__(self, criteria, min_samples_split=2, max_depth=100, n_feats=None):
        self.criteria = criteria
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        if self.n_feats:
            self.n_feats=min(X.shape[1],self.n_feats)
        else:
            self.n_feats=X.shape[1]
        # self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (
            depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split
            or y.size == 0
            or n_samples == 0
        ):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)

        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            #thresholds = np.unique(X_column)
            thresholds = np.linspace(0,1,100)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_impurity = self.criteria(y)
          # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self.criteria(y[left_idxs]), self.criteria(y[right_idxs])
        child_impurity = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_impurity - child_impurity
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
            return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        unique,class_count = np.unique(y,return_counts=True)
        most_common = unique[np.argmax(class_count)]
        return most_common


In [ ]:
clf = DecisionTree(criteria = gini,max_depth=10)
clf.fit(x_train, y_train)
y_pred_gini = clf.predict(x_test)

In [ ]:
#Accuracy for Gini Impurity
np.mean(y_pred_gini==y_test)